In [1]:
import pandas as pd
import useful_functions as use
from datetime import date

In [2]:
path = '../../Data/DataGov'
og_file = use.get_newest(path, 'PhysicianCompare')
phones_file = use.get_newest(path, 'DAC_National')
og = pd.read_csv(og_file, encoding='latin')
phones = pd.read_csv(phones_file, encoding='latin')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,17,25,29,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
phones.columns = [c.strip() for c in phones.columns.values]
og.columns = [c.strip() for c in og.columns.values]

In [8]:
all_= pd.merge(og, phones, on=['frst_nm', 'mid_nm', 'lst_nm', 'adrs_id', 'adr_ln_1',
       'adr_ln_2', 'cty', 'st', 'zip', 'org_pac_id', 'NPI'], how='outer')

In [9]:
this = all_[['phn_numbr_x','phn_numbr_y', 'adrs_id', 'NPI']]
this = this.fillna('None')

In [10]:
DICT_LIST = []
for row in this.itertuples():
    if row.phn_numbr_x =='None':
        phone = [row.phn_numbr_y]
    elif row.phn_numbr_y == 'None':
        phone = [row.phn_numbr_x]
    elif row.phn_numbr_y != row.phn_numbr_x:
        phone = [row.phn_numbr_x, row.phn_numbr_y]
    else:
        phone = [row.phn_numbr_x]
    for number in phone:
        new_dict = {
            'adrs_id': row.adrs_id,
            'phone': number,
            'NPI': row.NPI 
        }
        DICT_LIST.append(new_dict)

In [11]:
phoney = pd.DataFrame(DICT_LIST).drop_duplicates()

In [12]:
phoney = phoney[phoney.phone!='None']

In [13]:
gov = pd.merge(phones, phoney, on=['NPI', 'adrs_id'], how='left')

In [14]:
npi_me = use.get_npi_to_me()

In [15]:
gov.NPI = gov.NPI.astype(str)

In [17]:
gov.to_csv(f'../../Data/DataGov/All_Data_{og_file.split("File_")[1][0:6]}.csv', index=False)

KeyboardInterrupt: 